<a href="https://colab.research.google.com/github/Saarangagarwal/job-search-automator/blob/main/job_search_automator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import requests
import urllib
from lxml import html
import pandas as pd
import os
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
#extract jobs from indeed.ca and linkedin.ca
job_title = input("Enter the desired job title: ")
location = input("Enter the desired location: ")
number_of_jobs = input("Enter the number of desired results (approx. for indeed.ca only): ")

job_iterator = int(1)
if int(number_of_jobs) < 15:
  job_iterator = int(1)
elif int(number_of_jobs)%15 != 0:
  job_iterator = int((int(number_of_jobs)/15)+1)
else:
  job_iterator = int(int(number_of_jobs)/15)

jobs_list = {}
all_titles = []
all_companies = []
all_links = []
linkedin_jobs = []
a_all_titles = []
a_all_companies = []
a_all_links = []

def extract_jobs_indeed(job_title, location, jobs_list, iterator):
  start = (iterator)*10;
  setVariables = {'q' : job_title, 'l' : location, 'fromage' : 'last', 'sort' : 'date', 'start': start}
  url = ('https://www.indeed.ca/jobs?' + urllib.parse.urlencode(setVariables))
  print(url)
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  job_soup = soup.find(id="resultsCol")

  job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')

  cols = []
  extracted_info = []

  #getting job titles
  titles = []
  cols.append('Titles')
  for job_elem in job_elems:
      title_elem = job_elem.find('h2', class_='title')
      extr_title = title_elem.text.strip()
      titles.append(extr_title)
  extracted_info.append(titles) 

  #getting the names of the companies
  companies = []
  cols.append('Companies')
  for job_elem in job_elems:
      company_elem = job_elem.find('span', class_='company')
      extr_company = company_elem.text.strip()
      companies.append(extr_company)
  extracted_info.append(companies)

  #getting indeed links
  links = []
  cols.append('Links')
  for job_elem in job_elems:
      link = job_elem.find('a')['href']
      extr_link = 'www.indeed.ca' + link
      links.append(extr_link)
  extracted_info.append(links)

  #populating jobs list
  for j in range(len(cols)):
      jobs_list[cols[j]] = extracted_info[j]


def extract_jobs_linkedin(job_title, location):
  url = "https://www.linkedin.com/jobs/search?keywords=" + job_title + "&location=" + location + "&trk=homepage-jobseeker_jobs-search-bar_search-submit&position=2&pageNum=0"
  page = requests.get(url).text
  soup = BeautifulSoup(page, 'lxml')
  job_soup = soup.find_all('li', class_="result-card job-result-card result-card--with-hover-state")

  for j in job_soup:
    link = j.find('a', class_="result-card__full-card-link")
    jobTitle = j.find('h3', class_="result-card__title job-result-card__title").text
    employer = j.find('a', class_="result-card__subtitle-link job-result-card__subtitle-link").text

    jobList = [jobTitle, employer, link.get('href')]
    linkedin_jobs.append(jobList)
    linkedin_jobs.sort(key=lambda x: x[2], reverse=True)

    for l_j in linkedin_jobs:
      a_all_titles.append(l_j[0])
      a_all_companies.append(l_j[1])
      a_all_links.append(l_j[2])

  linkedin_job_summary = {
      "Titles": a_all_titles,
      "Companies": a_all_companies,
      "Links": a_all_links
  }

  return linkedin_job_summary
  


print("Searching for jobs on indeed.ca......")
for iterator in range(job_iterator):
  extract_jobs_indeed(job_title, location, jobs_list, iterator)
  itr_titles = jobs_list.get('Titles')
  itr_companies = jobs_list.get('Companies')
  itr_links = jobs_list.get('Links')
  for t in itr_titles:
    all_titles.append(t)
  for c in itr_companies:
    all_companies.append(c)
  for l in itr_links:
    all_links.append(l)

indeed_job_summary = {
    "Titles": all_titles,
    "Companies": all_companies,
    "Links": all_links
}

print(" ")
print("Searching for jobs on linkedin.com......")
print("Loading.....")
linkedin_results = extract_jobs_linkedin(job_title, location)

#getting current time
now = datetime.now()

year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
time = now.strftime("%H:%M:%S")
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

#generating excel file for the obtained job list
jobs = pd.DataFrame(indeed_job_summary)
jobs.to_excel("indeed_results_" + date_time.replace(" ", "").replace("/", "_").replace(":", "_") + ".xls")

#generating excel file for the obtained job list
jobs = pd.DataFrame(linkedin_results)
jobs.to_excel("linkedin_results_" + date_time.replace(" ", "").replace("/", "_").replace(":", "_") + ".xls")

print(" ")
print('Approximately {} job postings retrieved from indeed.ca. Stored in {}.'.format(number_of_jobs, "indeed_results_" + date_time.replace(" ", "").replace("/", "_").replace(":", "_") + ".xls"))
print('Job postings retrieved successfully from linkedin.com. Stored in {}.'.format("linkedin_results_" + date_time.replace(" ", "").replace("/", "_").replace(":", "_") + ".xls"))

